**Run all the cells below to make sure everything is working and ready to go. All cells should run without error.**

### Test Matplotlib and Plotting

In [11]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2 as cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import tensorflow as tf
from scipy import signal
import os
import math
from numpy.polynomial import Polynomial as P


PREV_LEFT_X1 = None
PREV_LEFT_X2 = None
PREV_RIGHT_X1 = None
PREV_RIGHT_X2 = None

BASE_IMG = None
CANNY_IMG = None

%matplotlib inline

In [12]:
src = np.float32([[760,450],[1130,650],[280,650],[560,450]])
dst = np.float32([[850,0],[850,600],[0,600],[0,0]])
M = cv2.getPerspectiveTransform(src,dst)
M_inv = cv2.getPerspectiveTransform(dst,src)


#define a birdview function to transform the area of interest to get a bird view of the lane
def birdview(img):
    dst1 = cv2.warpPerspective(img,M,(850,600),flags=cv2.INTER_LINEAR)
    return dst1

#define the sobel filter detect the edge of the lane 
def sobel_thresh (image, thresh_g_min,thresh_g_max,thresh_dir_min,thresh_dir_max):
    
    image = cv2.GaussianBlur(image,(5,5),0)
    
    image_x = cv2.Sobel(image,cv2.CV_64F,1,0, 5)
    image_y = cv2.Sobel(image,cv2.CV_64F,0,1, 5)
    
    abs_x = np.abs(image_x)
    abs_y = np.abs(image_y)
    
    img_g = np.sqrt(image_x**2+image_y**2)
    img_dir = np.arctan2(abs_y, abs_x)
    img_g = np.uint8(img_g*255/np.max(img_g))
    #img_dir = np.uint8(img_dir*255/np.max(img_dir))
    
    s_dir = np.zeros_like(img_dir)
    s_dir[(img_dir >= thresh_dir_min) & (img_dir <= thresh_dir_max)] = 1
    
    s_g = np.zeros_like(img_g)
    s_g[(img_g >= thresh_g_min) & (img_g <= thresh_g_max)] = 1
    
    #th1,img_g = cv2.threshold(img_g, thresh, max_value, cv2.THRESH_BINARY);
    #th1,img_dir = cv2.threshold(img_dir, thresh_dir, max_value, cv2.THRESH_BINARY);
    
    #img = cv2.bitwise_and(img_g, img_dir)
    
    binary_output = np.zeros_like(img_g)
    binary_output[(s_dir == 1)&(s_g == 1)]=1
    
    return binary_output

#define the function to run the sobel filter on the l and s channel of the image 
def hlv_sobel(img):
    h, l, s = cv2.split(cv2.cvtColor(img, cv2.COLOR_RGB2HLS))
    #img_h = sobel_thresh(h,50,255,0.5,1.2) 
    #plt.imshow(255*img_g, cmap ='gray')
    #print(img_h)
    #print(np.shape(img_h))
    img_s = sobel_thresh(s,35,255,1,120)
    img_l = sobel_thresh(l,35,255,1,120)
    image_hsv_sobel = cv2.bitwise_and(img_s,img_l)
    return image_hsv_sobel

#define s color mask to find the lane, the color mask is robut to shadow
def image_mask(image):

    image_HSV = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)

    #print(image_HSV[450,:,0])

    #use a for loop to normalize the image_HSV in Range 0 to 255
    #for i in range(3):
    #image_HSV[:,:,i] = np.uint8(image_HSV[:,:,i]*255/np.max(image_HSV[:,:,i]))

    #print(image_HSV[:,:,1], np.max(image_HSV[:,:,2]))

    yellow_hsv_low = np.array([ 10, 100, 100],np.uint8)
    yellow_hsv_high = np.array([ 80, 255, 255],np.uint8)

    white_hsv_low = np.array([20, 0, 180],np.uint8)
    white_hsv_high = np.array([255,  100, 255],np.uint8)


    #image_HSV = np.uint8(image_HSV*255/np.max(image_HSV))

    res_mask_yellow = cv2.inRange(image_HSV,yellow_hsv_low,yellow_hsv_high)
    res_yellow = cv2.bitwise_and(image,image, mask = res_mask_yellow)
 
    #print(image_HSV[200,:,0])
    #print(image_HSV[200,:,1])
    #print(image_HSV[200,:,2])
    res_mask_white = cv2.inRange(image_HSV,white_hsv_low,white_hsv_high)
    res_white = cv2.bitwise_and(image,image, mask = res_mask_white)
    
    #combine the yellow and white masked images
    mask_lane = cv2.bitwise_or(res_yellow,res_white)
    mask_lane_g = cv2.cvtColor(mask_lane, cv2.COLOR_RGB2GRAY)
    #Lane = np.zeros_like(img_s)
    #Lane[(mask_lane_g>=.5)|(image_hsv_sobel>=.5)]=1
    return mask_lane_g

#combine the image from sobel filter and the color mask
def CombineImage(image_sobel,image_mask):
    Lane = np.zeros_like(image_sobel)
    #Lane[(image_sobel>=.5)|(image_mask>=.5)]=1
    Lane = image_mask
    warp = cv2.warpPerspective(Lane, M_inv, (1280, 720))
    #mask = np.array
    #mask = 0*mask
    
    #Lane = cv2.bitwise_and(Lane,Lane,mask = mask)
    
    #Lane = cv2.bitwise_and()
    return warp




In [13]:
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def slope(line):
    return (float(line[3]) - line[1]) / (float(line[2]) - line[0])

#define the the draw_line function to draw the line back on the image 
def draw_lines(img, lines, color=[255, 0, 0], thickness=7):
    global PREV_LEFT_X1, PREV_LEFT_X2, PREV_RIGHT_X1, PREV_RIGHT_X2
    left_x = []
    left_y = []
    right_x = []
    right_y = []

    for line in lines:
        line = line[0]
        s = slope(line)

        if 0.3 > s > -0.3:
            continue

        if s < 0:
            if line[0] > img.shape[1] / 2 + 60:
                continue

            left_x += [line[0], line[2]]
            left_y += [line[1], line[3]]
            # cv2.line(img, (int(line[0]), int(line[1])), (int(line[2]), int(line[3])), [0, 0, 255], thickness)
        else:
            if line[0] < img.shape[1] / 2 - 20:
                continue

            right_x += [line[0], line[2]]
            right_y += [line[1], line[3]]
            # cv2.line(img, (int(line[0]), int(line[1])), (int(line[2]), int(line[3])), [255, 255, 0], thickness)

    y1 = img.shape[0]
    y2 = img.shape[0] / 2 + 90

    if len(left_x) <= 1 or len(right_x) <= 1:
        if PREV_LEFT_X1 is not None:
            cv2.line(img, (int(PREV_LEFT_X1), int(y1)), (int(PREV_LEFT_X2), int(y2)), color, thickness)
            cv2.line(img, (int(PREV_LEFT_X2), int(y1)), (int(PREV_RIGHT_X2), int(y2)), color, thickness)
        return

    left_poly = P.fit(np.array(left_x), np.array(left_y), 1)
    right_poly = P.fit(np.array(right_x), np.array(right_y), 1)

    left_x1 = (left_poly - y1).roots()
    right_x1 = (right_poly - y1).roots()

    left_x2 = (left_poly - y2).roots()
    right_x2 = (right_poly - y2).roots()

    if PREV_LEFT_X1 is not None:
        left_x1 = PREV_LEFT_X1 * 0.7 + left_x1 * 0.3
        left_x2 = PREV_LEFT_X2 * 0.7 + left_x2 * 0.3
        right_x1 = PREV_RIGHT_X1 * 0.7 + right_x1 * 0.3
        right_x2 = PREV_RIGHT_X2 * 0.7 + right_x2 * 0.3

    PREV_LEFT_X1 = left_x1
    PREV_LEFT_X2 = left_x2
    PREV_RIGHT_X1 = right_x1
    PREV_RIGHT_X2 = right_x2

    cv2.line(img, (int(left_x1), int(y1)), (int(left_x2), int(y2)), color, thickness)
    cv2.line(img, (int(right_x1), int(y1)), (int(right_x2), int(y2)), color, thickness)


#hough_lines function to find the lines in the processed image
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap,Oimg):
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    #print(lines[0],np.shape(lines))
    draw_lines(Oimg, lines)
    return Oimg

In [14]:
#this function process every image with the helper functions defined above and retur the image with lanes highlighted
def image_processing(img):
    image_bv = birdview(img)
    image_sobel = hlv_sobel(image_bv)
    img_mask = image_mask(image_bv)
    image_combined = CombineImage(image_sobel, img_mask)
    lines = hough_lines(image_combined, 1, np.pi/180, 1, 3, 5,img)
    #final = image_final(image_combined, img)
    return lines

In [15]:
PREV_LEFT_X1 = None
PREV_LEFT_X2 = None
PREV_RIGHT_X1 = None
PREV_RIGHT_X2 = None

BASE_IMG = None
CANNY_IMG = None

src = np.float32([[760,450],[1130,650],[280,650],[560,450]])
dst = np.float32([[850,0],[850,600],[0,600],[0,0]])
M = cv2.getPerspectiveTransform(src,dst)
M_inv = cv2.getPerspectiveTransform(dst,src)


new_clip_output = 'test_output.mp4'
test_clip = VideoFileClip("test.mp4")
new_clip = test_clip.fl_image(image_processing)#NOTE: this function expects color images!!
%time new_clip.write_videofile(new_clip_output, audio=False)

[MoviePy] >>>> Building video test_output.mp4
[MoviePy] Writing video test_output.mp4
[MoviePy] Done.                                                        
[MoviePy] >>>> Video ready: test_output.mp4 

CPU times: user 57.9 s, sys: 9.77 s, total: 1min 7s
Wall time: 36 s


HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))

